# **Preorocessing Program**

In [2]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

stopwords[:10]

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해']]

## **Develop Preprocessing Program**

In [2]:
def arguments():
    
    
    ### 1) 매체 선택
    media_list = ['매일경제', '아시아경제', '삼프로TV', '슈카월드', '한국경제TV']
    media_name = str(input('***매체명 입력(매일경제/아시아경제/삼프로TV/슈카월드/한국경제TV) :'))
    while media_name not in media_list:
        media_name = str(input('***매체명 다시 입력(매일경제/아시아경제/삼프로TV/슈카월드/한국경제TV) :'))
        if media_name in media_list:
            break
    if media_name == '매일경제':
        globals()['craw_media'] = 'maeil_news_craw'
    elif media_name == '아시아경제':
        globals()['craw_media'] = 'asia_news_craw'
    elif media_name == '삼프로TV':
        globals()['craw_media'] = 'youtube_sampro'
    elif media_name == '슈카월드':
        globals()['craw_media'] = 'youtube_suka'
    else:
        globals()['craw_media'] = 'youtube_hk'
    
    
    ### 2) date 지정
    
    ## 2-1) 시작 날짜
    globals()['start_date'] = str(input('***시작 날짜(YYYY-MM-DD) :'))
    while len(globals()['start_date']) != 10:
        start_date = str(input('***시작 날짜 다시 입력(YYYY-MM-DD) :'))
        if len(globals()['start_date']) == 10:
            break
    globals()['start_date'] = globals()['start_date'].replace('/', '-')
    globals()['start_date'] = globals()['start_date'].replace('.', '-')
    
    ## 2-2) 종료 날짜
    globals()['end_date'] = str(input('***종료 날짜(YYYY-MM-DD) :'))
    while len(globals()['end_date']) != 10:
        globals()['end_date'] = str(input('***종료 날짜 다시 입력(YYYY-MM-DD) :'))
        if len(end_date) == 10:
            break
    globals()['end_date'] = globals()['end_date'].replace('/', '-')
    globals()['end_date'] = globals()['end_date'].replace('.', '-')
    
    ## 2-3) if 시작 날짜 < 종료 날짜: ...;;
    if globals()['start_date'] > globals()['end_date']:
        globals()['start_date'], globals()['end_date'] = globals()['end_date'], globals()['start_date']
    else:
        pass
    
    
    return globals()['craw_media'], globals()['start_date'], globals()['end_date']

In [3]:
#*** 아직 YouTube 채널 크롤링 데이터는 별도의 전처리 코드 작성 필요 ***#
def media_stock_prediction(craw_media, start_date, end_date):
    #### 1. Read Data
    
    
    ### 1) KOSELF 감성 어휘 사전
    #*** 추후에 감성사전도 DB 연결해서 사용하도록 코드 변경 필요 ***#
    with open('KOSELF_pos.txt', encoding='utf-8') as pos:
        positive = pos.readlines()
    positive = [pos.replace('\n', '') for pos in positive]
    with open('KOSELF_neg.txt', encoding='utf-8') as neg:
        negative = neg.readlines()
    negative = [neg.replace('\n', '') for neg in negative]
    
    
    ### 2) News Data from DB
    db = pymysql.connect(user='root',
                         passwd='0808',
                         host='127.0.0.1',
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)
    
    ## 2-1) 전체 종목 뉴스 데이터
    corp_list = ['samsung', 'hyundai', 'lg', 'sk', 'celltrion']
    stock_num_list = ['005930', '005380', '051910', '000660', '068270']
    
    ## 2-2) DB의 date 컬럼과 형태 통일
    start_date = start_date.replace('-', '')
    end_date = end_date.replace('-', '')
    for i in range(len(corp_list)):
        sql = "select * from {0}_{1} where (length(date)=10) and (date between {2}00 and {3}23)".format(craw_media, stock_num_list[i], start_date, end_date)
        cursor.execute(sql)
        result = cursor.fetchall()

        # DataFrame으로 변경
        globals()[corp_list[i]] = pd.DataFrame(result)
    
    db.close()   # 메모리 절약
    
    ## 2-3) 날짜와 시간 구분
    for i in range(len(corp_list)):
        globals()[corp_list[i]].rename(columns={'date': 'datetime'}, inplace=True)

        # DataFrame 형태를 통일하기 위해 date 컬럼 추가
        globals()[corp_list[i]]['date'] = globals()[corp_list[i]]['datetime'].str[0:4] + '-' + globals()[corp_list[i]]['datetime'].str[4:6] + '-' + globals()[corp_list[i]]['datetime'].str[6:8]
        globals()[corp_list[i]]['date'] = pd.to_datetime(globals()[corp_list[i]]['date'])

        # 결측치 제거 → 데이터 로드 시 완료했기 때문에 그다지 필요하지 않은 과정
        globals()[corp_list[i]] = globals()[corp_list[i]].dropna()

        # 시간순으로 정렬
        globals()[corp_list[i]].sort_values('datetime', inplace=True)
        globals()[corp_list[i]].reset_index(inplace=True, drop=True)
    
    
    ### 3) FinanceDataReader
    # 종료 날짜는 현재 시각을 기준으로
    end_date = datetime.datetime.now().strftime("%Y%m%d")
    for i in range(len(corp_list)):
        globals()['stock_' + corp_list[i]] = fdr.DataReader(stock_num_list[i], start=start_date, end=end_date).reset_index()
    
    
    ### 4) Holidays
    db = pymysql.connect(user='root',
                         passwd='0808',
                         host='127.0.0.1',
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)

    # 4-1) 주말 및 공휴일 데이터
    sql = "select * from holidays"
    cursor.execute(sql)
    result = cursor.fetchall()
    
    # DataFrame으로 변경
    globals()['holidays'] = pd.DataFrame(result)
    
    db.close()   # 메모리 절약
    
    # 4-2) date 컬럼을 날짜 형식으로 변경
    globals()['holidays']['date'] = pd.to_datetime(holidays['date'])
    
    
    ### 5) Stop Words
#     #*** 추후에 Stop Words도 DB 연결해서 사용하도록 코드 변경 필요 ***#
#     with open('stopwords-ko.txt', encoding='utf-8') as sw:
#         globals()['stop_words'] = sw.readlines()
#     globals()['stop_words'] = [sw.replace('\n', '') for sw in stop_words]
    # GitHub로부터 Stop Words 로드
    stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt")
    # Stop Words List에 각 매체명 추가
    except_media_list = ['매일경제', '매일', '경제', 'maeil', 'MK', 'mk',
                         '아시아경제', '아시아', 'Asia', 'ASIA', 'asia',
                         '삼프로TV', '삼프로', 'TV',
                         '슈카월드', '슈카', '월드'
                         '한국경제TV', '한국']
    for word in [except_media_list]:
        stopwords.append(word)
    
    
    
    
    #### 2. Preprocessing
    '''감성 어휘 사전 : negative / positive
       뉴스 데이터 : samsung / hyundai / lg / sk
       주식 데이터 : stock_samsung / stock_hyundai / stock_lg / stock_sk
       공휴일 데이터 : holidays'''
    
    
    ### 1) 뉴스 데이터 날짜 조정
    
    ## 1-1)업로드 시각 컬럼 추가
    for i in range(len(corp_list)):
        globals()[corp_list[i]]['time'] = globals()[corp_list[i]]['datetime'].str[-2:]
    
    ## 1-2) 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']

    for i in range(len(corp_list)):
        for j in range(len(globals()[corp_list[i]]['time'])):
            if globals()[corp_list[i]]['time'][j] in after_market:
                globals()[corp_list[i]]['date'][j] += datetime.timedelta(1)
            else:
                pass
    
    ## 1-3) 텍스트 전처리
    # \n, \t, \r 제거
    for i in range(len(corp_list)):
        for j in range(len(globals()[corp_list[i]]['text'])):
            globals()[corp_list[i]]['text'][j] = globals()[corp_list[i]]['text'][j].replace('[\n|\t|\r]', '')
#     # text 컬럼의 Stop Words 제거
#     for i in range(len(corp_list)):
#         globals()[corp_list[i]]['except_stopwords'] = 0
#         for j in range(len(globals()[corp_list[i]]['text'])):            
#             hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                         # 정규 표현식  → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
#             result = hangeul.sub('', globals()[corp_list[i]]['text'][j])   # 위에 설정한 hangeul 규칙을 text에 적용
#             okt = Okt()                                                    # 형태소 추출
#             nouns = okt.nouns(result)
#             nouns = [x for x in nouns if len(x) > 1]                       # 한 글자 키워드 제거
#             nouns = [x for x in nouns if x not in stopwords]               # 불용어 제거
            
#             corpus = " ".join(nouns)                                       # List를 String으로 변환
#             globals()[corp_list[i]]['except_stopwords'][j] = corpus
    
        
    ### 2) 주말 및 공휴일 제외
    
    ## 2-1) 주말 및 공휴일만 추출
    market_closed = globals()['holidays'][globals()['holidays']['holiday']=="O"].reset_index(drop=True)
    
    ## 2-3) 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    ## 2-4) iteration limit 조정
    limit_number = 15000
    sys.setrecursionlimit(limit_number)
    
    ## 2-5) 휴장일 제외 함수 적용
    # 주말 및 공휴일 제외 함수
#     def stock_market_closed(df):
#         for i in range(len(df['date'])):
#             if df['date'][i] in market_closed_list:
#                 df['date'][i] += datetime.timedelta(1)
#                 stock_market_closed(df)
#             else:
#                 pass
#         return df
    
#     for i in range(len(corp_list)):
#         stock_market_closed(globals()[corp_list[i]])
    for i in range(len(corp_list)):
        while len(globals()[corp_list[i]][globals()[corp_list[i]]['date'].isin(market_closed_list)]['date']) != 0:
            for j in globals()[corp_list[i]][globals()[corp_list[i]]['date'].isin(market_closed_list)]['date'].index:
                globals()[corp_list[i]]['date'][j] += datetime.timedelta(1)
    
    
    
        
    #### 3. Sentiment Analysis
    
    
#     ### 1) 종목별 긍부정 Score 계산
#     for i in range(len(corp_list)):
#         globals()[corp_list[i]]['score'] = 0
#         tokenizer = Okt()

#         for x in range(len(globals()[corp_list[i]]['date'])):
#             score = 0
#             num = tokenizer.nouns(globals()[corp_list[i]]['text'][x])
#             for y in num:
#                 # KOSELF 감성 어휘 사전
#                 if y in positive:
#                     score += 1
#                 elif y in negative:
#                     score -= 1
#                 else:
#                     score = score

#             globals()[corp_list[i]]['score'][x] = score
    
    
    ### 2) 주식가격 데이터와 결합
    corp_label_list = []
    for i in range(len(corp_list)):
        
        ## 2-1) 결합
        globals()[corp_list[i] + '_label'] = pd.merge(globals()[corp_list[i]], globals()['stock_' + corp_list[i]], how='left', left_on='date', right_on='Date')
        globals()[corp_list[i] + '_label'].drop('Date', axis=1, inplace=True)
        
        ## 2-2) UpDown과 Extremely_Changed(Change 상하위 5%) 컬럼 생성
        # 주식 매매 수수료 평균 : 0.1% 정도(?) → 0을 추가해도 1, -1만 나옴
        globals()[corp_list[i] + '_label']['UpDown'] = np.where((globals()[corp_list[i] + '_label']['Close']*globals()[corp_list[i] + '_label']['Change'])>(globals()[corp_list[i] + '_label']['Close']*0.001), 1,
                                                                np.where((globals()[corp_list[i] + '_label']['Close']*globals()[corp_list[i] + '_label']['Change']<globals()[corp_list[i] + '_label']['Close']*0.001), -1, 0))
#         # 단순히 Change가 (+), 0, (-)인지에 따라 각각 1, 0, -1
#         globals()[corp_list[i] + '_label']['UpDown'] = np.where(globals()[corp_list[i] + '_label']['Change']>0, 1,
#                                                                 np.where(globals()[corp_list[i] + '_label']['Change']<0, -1, 0))
        globals()[corp_list[i] + '_label']['Extremely_Changed'] = np.where((globals()[corp_list[i] + '_label']['Change']>globals()[corp_list[i] + '_label']['Change'].quantile(.95)) & (globals()[corp_list[i] + '_label']['Change']>0), 1,
                                                                           np.where((globals()[corp_list[i] + '_label']['Change']<globals()[corp_list[i] + '_label']['Change'].quantile(.05)) & (globals()[corp_list[i] + '_label']['Change']<0), -1, 0))
        
        ## 2-3) List에 추가
        corp_label_list.append(globals()[corp_list[i] + '_label'])
        
        ## 2-4) text 컬럼의 NaN 제거
        globals()[corp_list[i] + '_label'].dropna(axis=0, inplace=True)
    
    
    ### 3) Tokenization 컬럼 추가
    for i in range(len(corp_list)):
        globals()[corp_list[i] + '_label']['Tokenization'] = 0
        rows = globals()[corp_list[i] + '_label'].shape[0]
        for j in range(rows):
            hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                                    # 정규 표현식 → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
            result = hangeul.sub('', globals()[corp_list[i] + '_label']['text'][j])   # 위에 설정한 hangeul 규칙을 text에 적용
            okt = Okt()                                                               # 형태소 추출
            nouns = okt.nouns(globals()[corp_list[i] + '_label']['text'][j])
            nouns = [x for x in nouns if len(x) > 1]                                  # 한 글자 키워드 제거
            nouns = [x for x in nouns if x not in stopwords]                          # 불용어 제거
            
            corpus = " ".join(nouns)                                                  # List를 String으로 변환
            globals()[corp_list[i] + '_label']['Tokenization'][j] = corpus
        
    
    ### 4) 전체 종목 DataFrame 통합
    globals()['total_label'] = pd.concat(corp_label_list, axis=0)

In [3]:
holidays = pd.read_csv('holidays.csv')

holidays

,date,year,month,day,dd,explanation,weekend,holiday
0,2018-01-01,2018,1,1,Mon,����,NaN,O
1,2018-01-02,2018,1,2,Tue,NaN,NaN,NaN
2,2018-01-03,2018,1,3,Wed,NaN,NaN,NaN
3,2018-01-04,2018,1,4,Thu,NaN,NaN,NaN
4,2018-01-05,2018,1,5,Fri,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1821,2022-12-27,2022,12,27,Tue,NaN,NaN,NaN
1822,2022-12-28,2022,12,28,Wed,NaN,NaN,NaN
1823,2022-12-29,2022,12,29,Thu,NaN,NaN,NaN
1824,2022-12-30,2022,12,30,Fri,NaN,NaN,NaN


In [37]:
holidays['date'] = pd.to_datetime(holidays['date'])

In [6]:
market_closed = holidays[holidays.holiday=='O'].reset_index(drop=True)
market_closed_list = list(market_closed['date'])

In [27]:
db = pymysql.connect(user='root',
                     passwd='0808',
                     host='127.0.0.1',
                     db='proj',
                     charset='utf8')

cursor = db.cursor(pymysql.cursors.DictCursor)

In [28]:
sql = "select * from maeil_news_craw_005930 where (length(date)=10) and (date between 2018010100 and 2018123123)"
cursor.execute(sql)
result = cursor.fetchall()

samsung = pd.DataFrame(result)

db.close()

In [29]:
samsung.head(3)

,st_n,st_cd,news,date,title,url,text
0,삼성전자,005930,매일경제,2018062215,숨고르는 삼성전자…주가 모멘텀 3분기부터,http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n 삼성전자 주가가 힘쓰지 못한 채 지지부진한 모습을 보이고 있다. 액면분...
1,삼성전자,005930,매일경제,2018062213,"삼성전자, 차세대 배터리 기술 주제 `전고체전지 포럼` 개최",http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n\n참석자들이 이세희 미국 콜로라도대 교수 강의를 경청하고 있다. [사진...
2,삼성전자,005930,매일경제,2018062204,[빅데이터로 본 제테크] 대내외 악재에 고전하는 `국민株` 삼성전자,http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n삼성전자 서초사옥. [매경 DB]\n 지난달 '황제주'에서 '국민주'로 ...


In [50]:
samsung

,st_n,st_cd,news,datetime,title,url,text,date,time
0,삼성전자,005930,매일경제,2018010100,스마트베타ETF 고공행진 새해도 이어질까,http://news.mk.co.kr/newsRead.php?year=2018&no=29,수수료가 싼 상장지수펀드(ETF)에 펀드매니저가 종목을 고르는 액티브 펀드 특성을...,2018-01-02,00
1,삼성전자,005930,매일경제,2018010100,"동원산업·휠라코리아 ""생큐 트럼프""",http://news.mk.co.kr/newsRead.php?year=2018&no=30,■ 2018 美법인세인하 수혜株 올해 1월부터 법인세 인하를 골자로 한 미국 세제...,2018-01-02,00
2,삼성전자,005930,매일경제,2018010100,매경춘추 새 필진과 새해 아침을 여세요,http://news.mk.co.kr/newsRead.php?year=2018&no=18,명품 에세이 매경춘추가 새해를 맞아 새 필진을 소개합니다. 유독 다사다난했던 20...,2018-01-02,00
3,삼성전자,005930,매일경제,2018010104,[2018 경제기상도] `1월효과` 기대 커진 증시…반도체·바이오株 계속 달릴듯,http://news.mk.co.kr/newsRead.php?year=2018&no...,지난해 연말 조정장세는 올해 초 코스피와 코스닥시장 '1월 효과'의 기폭제가 될 ...,2018-01-02,04
4,삼성전자,005930,매일경제,2018010104,"[2018 국내 업종별 전망] 가전, OLED TV `올림픽·월드컵 겹호재`",http://news.mk.co.kr/newsRead.php?year=2018&no...,지난해 글로벌 TV 수요가 전반적으로 감소한 가운데 새해 가전시장에서는 프리미엄 ...,2018-01-02,04
...,...,...,...,...,...,...,...,...,...
4360,삼성전자,005930,매일경제,2018123108,"[Hot-Line] ""1월 최선호주는 삼성전기·삼성SDI·LG전자""",http://news.mk.co.kr/newsRead.php?year=2018&no...,"대신증권이 새해 1월 최선호주로 삼성전기와 삼성SDI, LG전자를 제시했다.대신증권...",2019-01-02,08
4361,삼성전자,005930,매일경제,2018123108,[2019 IT 트렌드②] `5G 원년` 20배 빠른 세상 본격 열린다,http://news.mk.co.kr/newsRead.php?year=2018&no...,내년 통신 산업 핵심 키워드는 5세대 이동통신 '5G'다. 지난 1일 우리나라가 ...,2019-01-02,08
4362,삼성전자,005930,매일경제,2018123115,"中 스마트폰 프리미엄도 강하다…화웨이, 점유율 첫 10%대 진입",http://news.mk.co.kr/newsRead.php?year=2018&no...,(왼쪽부터) 삼성전자 `갤럭시노트9` 화웨이 `P20` 애플 `아이폰XS` [사진=...,2019-01-02,15
4363,삼성전자,005930,매일경제,2018123116,[기해년 주요 일정] 3·1운동·임정 100주년…트럼프 연두교서 `G2 발언` 촉각,http://news.mk.co.kr/newsRead.php?year=2018&no...,◆ 2019 신년기획 ◆ 올해는 대한민국 임시정부 수립과 3·1절을 기점으로 한 ...,2019-01-02,16


In [34]:
samsung.rename(columns={'date': 'datetime'}, inplace=True)
samsung['date'] = samsung['datetime'].str[0:4] + '-' + samsung['datetime'].str[4:6] + '-' + samsung['datetime'].str[6:8]
samsung['date'] = pd.to_datetime(samsung['date'])
samsung = samsung.dropna()
samsung.sort_values('datetime', inplace=True)
samsung.reset_index(inplace=True, drop=True)

In [36]:
stock_samsung = fdr.DataReader('005930', start='2018-01-01', end='2021-09-15').reset_index()

stock_samsung

,Date,Open,High,Low,Close,Volume,Change
0,2018-01-02,51380,51400,50780,51020,169485,0.001177
1,2018-01-03,52540,52560,51420,51620,200270,0.011760
2,2018-01-04,52120,52180,50640,51080,233909,-0.010461
3,2018-01-05,51300,52120,51200,52120,189623,0.020360
4,2018-01-08,52400,52520,51500,52020,167673,-0.001919
...,...,...,...,...,...,...,...
910,2021-09-09,76400,76600,75000,75300,17600770,-0.013106
911,2021-09-10,75300,75600,74800,75300,10103212,0.000000
912,2021-09-13,75200,76300,75100,76300,11397775,0.013280
913,2021-09-14,77100,77700,76600,76600,18167057,0.003932


In [38]:
samsung['time'] = samsung['datetime'].str[-2:]

In [39]:
after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']
for i in range(len(samsung['time'])):
    if samsung['time'][i] in after_market:
        samsung['date'][i] += datetime.timedelta(1)
    else:
        pass

In [40]:
samsung['text'] = samsung['text'].str.replace('[\n|\t|\r]', '')

In [41]:
while len(samsung[samsung['date'].isin(market_closed_list)]['date']) != 0:
    for i in samsung[samsung['date'].isin(market_closed_list)]['date'].index:
        samsung['date'][i] += datetime.timedelta(1)

In [43]:
samsung[samsung['date']=='2021-05-05']

,st_n,st_cd,news,datetime,title,url,text,date,time


In [46]:
corp_list = ['samsung', 'hyundai']

In [51]:
type(globals()[corp_list[0]].shape[0])

int